In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from mplex.cm import get_transparent_cmap
from mplex.colors import change_hsv
from mplex.text import set_text_outline
from bino_utils.plotting import BrainGrid
from bino_utils import Atlas
from bg_space_extra import AnatomicalPoints

atlas = Atlas()

In [2]:
def get_trans_cmap(c, n=256):
    from matplotlib.colors import ListedColormap, to_rgb

    return ListedColormap(
        np.column_stack((np.tile(to_rgb(c), (n, 1)), np.linspace(0, 1, n)))
    )

In [3]:
data_dir = Path("../../data/ocular_types")
out_dir = Path("outputs")
fmt = ".pdf"
out_dir.mkdir(exist_ok=True)
points = AnatomicalPoints.from_dataframe(pd.read_hdf(data_dir / "points.h5"))
df_regions = pd.read_hdf(data_dir / "regions.h5")

In [4]:
regions = (
    "mesencephalon (midbrain)",
    "prosencephalon (forebrain)",
    "rhombencephalon (hindbrain)",
)
bidx = atlas.is_points_in_structures(points, regions) & ~atlas.is_points_in_structures(
    points, "retina"
)
shift = {
    "pretectum (alar prosomere 1)": (-20, -18),
    "thalamus proper": (40, 10),
    "tectum": (-25, 25),
    "nucleus isthmi": (0, 20),
    "tegmentum (midbrain tegmentum)": (5, 0),
}
cmap_retina = get_transparent_cmap("gray_r", "w")

In [ ]:
bg = BrainGrid(atlas, w=100, bounds=dict(p=753), space_within=(4, 2))
bp = bg[0, 0]
bp.add_structures("root", n=1, ec=(0.7,) * 3, zorder=-1000, fc="w")
bp.add_structures("retina", method="mean", cmap=cmap_retina, zorder=-10000, vmax=1)
bp.scatter(
    points[bidx],
    rasterized=True,
    s=0.5,
    alpha=0.1,
    color=(0.2,) * 3,
    zorder=-100,
    marker=".",
    lw=0,
)

for i in df_regions.itertuples():
    cmap_i = get_trans_cmap(i.color)
    bp.add_structures(i.Index, method="mean", cmap=cmap_i, vmin=0, vmax=0.8)
    n = 2 if i.Index == "nucleus isthmi" else 1
    bp.add_structures(i.Index, method="max", ec=change_hsv(i.color), n=n)
    xy = np.array((i.zm, i.xm)) + shift.get(i.Index, (0, 0))
    bp["left"].add_text(
        *xy,
        i.acronym,
        transform="d",
        ha="c",
        va="c",
        zorder=1e6,
        color=change_hsv(i.color, v=0.7),
        outline_kwargs=dict(linewidth=1, foreground="w"),
        weight="bold",
    )

ret = bp.add_anatomical_axes(77, 332, 332, 753)

ax = bp["left"]
ax.plot(
    [165 - 50, 165 + 50],
    [1.04] * 2,
    transform=ax.get_xaxis_transform(),
    clip_on=False,
    color="k",
)

ax.add_text(
    165,
    1.04,
    "100 µm",
    ha="c",
    va="b",
    fontsize=7,
    color="k",
    pad=(0, 1),
    zorder=1000,
    clip_on=False,
    transform=ax.get_xaxis_transform(),
)
set_text_outline(ret["front"]["ylabel"])

if fmt:
    bg.grid.savefig((out_dir / "imaging_volume").with_suffix(fmt))